# Development of miscellaneous new features for polymerist

## Testing polymerist importability within environment

In [ ]:
import polymerist as ps

print(ps.__version__)
print(ps.pascal(5))

In [ ]:
from polymerist.mdtools import openfftools

openfftools.POLYMERIST_TOOLKIT_REGISTRY.registered_toolkits

In [ ]:
import polymerist as ps
from polymerist.genutils.importutils import pyimports, module_hierarchy

print(module_hierarchy(ps))

In [ ]:
import nglview

print(nglview.__version__)
nglview.demo()

In [ ]:
from polymerist.polymers.monomers import specification
from rdkit import Chem

smi = 'CCO-c1ccccc1-N=C=C'
mol1 = Chem.MolFromSmiles(smi)
display(mol1)

sma = specification.expanded_SMILES(smi, assign_map_nums=True)
exp_sma = specification.compliant_mol_SMARTS(sma)
mol2 = Chem.MolFromSmarts(sma)
display(mol2)


In [ ]:
import numpy as np
import pandas as pd

from openff.toolkit import Molecule, Topology, ForceField

offmol = Molecule.from_smiles(smi)
offmol.generate_conformers(n_conformers=1)
offmol.visualize(backend='nglview')

# Another thing